In [1]:
import os
import numpy as np
import pandas as pd

from arkane.main import Arkane
from arkane.pdep import PressureDependenceJob
from arkane.sensitivity import PDepSensitivity
from rmgpy.species import Species
from rmgpy.tools.canteraModel import get_rmg_species_from_user_species
from rmgpy.tools.uncertainty import Uncertainty
from rmgpy.exceptions import InvalidMicrocanonicalRateError, ModifiedStrongCollisionError
from rmgpy.quantity import Quantity

In [2]:
base_path = '/home/mjliu/Documents/new_sensitivity/sensitivity_related'
chem_path = os.path.join(base_path, 'chem_with_thermo_manually.inp')
dict_path = os.path.join(base_path, 'species_dictionary_PES.txt')

# Sensitivity of concentrations to thermo and kinetics

In [3]:
uncertainty = Uncertainty(output_directory=os.path.join(base_path, 'uncertainty'))
uncertainty.load_model(chem_path, dict_path)

In [4]:
# Sort reactants and products
for rxn in uncertainty.reaction_list:
    rxn.reactants.sort()
    rxn.products.sort()

In [25]:
he = Species(smiles='[He]')
c2h4 = Species(smiles='C=C')
c6h5 = Species(smiles='[c]1ccccc1')
c6h6 = Species(smiles='c1ccccc1')
styrene = Species(smiles='C=Cc1ccccc1')

mapping = get_rmg_species_from_user_species([he, c2h4, c6h5, c6h6, styrene], uncertainty.species_list)

initial_mole_fractions = {
    mapping[c6h5]: 1.8e+12,
    mapping[c2h4]: 4.83e+16,
    mapping[he]: 1.13e+17,
}

total_moles = sum(initial_mole_fractions.values())

for spc, val in initial_mole_fractions.items():
    initial_mole_fractions[spc] = val / total_moles

T = (607,'K')
P = (10,'torr')

termination_time = (0.01, 's')
sensitive_species=[mapping[styrene]]

In [26]:
# Perform the sensitivity analysis
uncertainty.sensitivity_analysis(
    initial_mole_fractions,
    sensitive_species,
    T,
    P,
    termination_time,
    sensitivity_threshold=1e-8,
    number=10,
    fileformat='.png',
)

At time 1.0000e-02 s, reached target termination time.


In [27]:
# Load sensitivity data
sens_data = {}
for species in sensitive_species:
    csv_file = os.path.join(uncertainty.output_directory, 'solver', 'sensitivity_1_SPC_{0}.csv'.format(species.index))
    dataframe = pd.read_csv(csv_file)
    sens_data[species] = dataframe

In [28]:
sens_data[sensitive_species[0]]

,Time (s),dln[C6H5C2H3]/dln[k1]: C2H4+C6H5=C2H3+C6H6,dln[C6H5C2H3]/dln[k2]: i2(+M)=i1(+M),dln[C6H5C2H3]/dln[k3]: i3(+M)=i1(+M),dln[C6H5C2H3]/dln[k4]: i4(+M)=i1(+M),dln[C6H5C2H3]/dln[k5]: i5(+M)=i1(+M),dln[C6H5C2H3]/dln[k6]: C2H4+C6H5(+M)=i1(+M),dln[C6H5C2H3]/dln[k7]: i3(+M)=i2(+M),dln[C6H5C2H3]/dln[k8]: i4(+M)=i2(+M),dln[C6H5C2H3]/dln[k10]: C2H4+C6H5(+M)=i2(+M),...,dln[C6H5C2H3]/dG[H],dln[C6H5C2H3]/dG[i1],dln[C6H5C2H3]/dG[i2],dln[C6H5C2H3]/dG[i3],dln[C6H5C2H3]/dG[i4],dln[C6H5C2H3]/dG[i5],dln[C6H5C2H3]/dG[i6],dln[C6H5C2H3]/dG[i7],dln[C6H5C2H3]/dG[i8],dln[C6H5C2H3]/dG[i9]
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1.000000e-15,-5.119743e-14,-1.849795e-28,-7.742943e-23,-1.024799e-20,-6.464989e-22,-1.899743e-12,-8.758213e-26,-1.461047e-25,-4.256518e-14,...,-1.945592e-27,1.149318e-24,1.593406e-28,-9.216901e-25,-1.381227e-24,-3.360660e-22,1.123501e-31,1.926576e-27,9.471327e-30,1.384901e-30
2,3.000000e-15,-9.751892e-14,-5.754917e-28,-2.408916e-22,-3.188264e-20,-2.011330e-21,-3.618548e-12,-2.724777e-25,-4.545479e-25,-8.107632e-14,...,-7.164721e-27,3.575657e-24,4.991442e-28,-2.867480e-24,-4.297151e-24,-1.045539e-21,4.137336e-31,7.094691e-27,3.487854e-29,5.099952e-30
3,7.000000e-15,-2.407177e-13,-3.690936e-27,-1.544966e-21,-2.044802e-19,-1.289973e-20,-8.932088e-12,-1.747545e-24,-2.915255e-24,-2.001302e-13,...,-5.192798e-26,2.293260e-23,3.219652e-27,-1.839068e-23,-2.755989e-23,-6.705594e-21,2.998630e-30,5.142042e-26,2.527903e-28,3.696308e-29
4,1.500000e-14,-5.159431e-13,-1.654395e-26,-6.925032e-21,-9.165438e-19,-5.782065e-20,-1.914460e-11,-7.833056e-24,-1.306708e-23,-4.289494e-13,...,-2.460395e-25,1.027913e-22,1.447234e-26,-8.243291e-23,-1.235321e-22,-3.005660e-20,1.420778e-29,2.436347e-25,1.197744e-27,1.751345e-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.494414e-03,-2.317852e-02,-4.039078e-03,-2.359550e-04,-1.817279e-07,-2.741625e-06,-4.291575e-01,-2.472703e-04,-1.571214e-07,-1.926419e-02,...,-5.085401e-03,9.903559e-03,3.376850e-03,-1.079847e-02,-5.322069e-06,2.208688e-06,1.138143e-07,5.039050e-03,3.187804e-06,4.795076e-06
448,9.744945e-03,-2.318594e-02,-4.207673e-03,-2.320289e-04,-1.790370e-07,-2.701119e-06,-4.228153e-01,-2.569175e-04,-1.635072e-07,-1.927027e-02,...,-5.197476e-03,9.942330e-03,3.517575e-03,-1.094122e-02,-5.392702e-06,2.176007e-06,1.207708e-07,5.149662e-03,3.230468e-06,4.923387e-06
449,9.995476e-03,-2.319316e-02,-4.377741e-03,-2.282079e-04,-1.764176e-07,-2.661692e-06,-4.166420e-01,-2.666374e-04,-1.699459e-07,-1.927617e-02,...,-5.309045e-03,9.974596e-03,3.659531e-03,-1.107974e-02,-5.461197e-06,2.144197e-06,1.278973e-07,5.259756e-03,3.273070e-06,5.051749e-06
450,1.000000e-02,-2.290799e-02,-4.326935e-03,-2.253335e-04,-1.742014e-07,-2.628257e-06,-4.114082e-01,-2.635315e-04,-1.679706e-07,-1.903915e-02,...,-5.245723e-03,9.852414e-03,3.617057e-03,-1.094588e-02,-5.395220e-06,2.117261e-06,1.264526e-07,5.197013e-03,3.233566e-06,4.991896e-06


# Sensitivity of rates to E0

In [10]:
arkane_input = os.path.join(base_path, 'input.py')

sensitivity_conditions = [[T, P]]

In [11]:
# Run arkane job so we can get the original objects
arkane = Arkane(input_file=os.path.join(base_path, 'input.py'), output_directory=base_path)
arkane.plot = False
# Run the main arkane job
arkane.execute()

Arkane execution initiated at Mon Oct  7 14:28:18 2019

################################################################
#                                                              #
# Automated Reaction Kinetics and Network Exploration (Arkane) #
#                                                              #
#   Version: 2.4.1                                             #
#   Authors: RMG Developers (rmg_dev@mit.edu)                  #
#   P.I.s:   William H. Green (whgreen@mit.edu)                #
#            Richard H. West (r.west@neu.edu)                  #
#   Website: http://reactionmechanismgenerator.github.io/      #
#                                                              #
################################################################

Databases not found. Making databases
Loading frequencies group database from /home/mjliu/Code/RMG-database/input/statmech/groups...
Loading transport library from OneDMinN2.py in /home/mjliu/Code/RMG-database/input/transport/li

/home/mjliu/Code/RMG-Py/rmgpy/pdep/network.py:276: RuntimeWarning: invalid value encountered in double_scalars
  Keq0 = K[t, p, j, i] / K[t, p, i, j]


Using 250 grains from -204.18 to 169.67 kJ/mol in steps of 1.50 kJ/mol to compute the k(T,P) values at 319.579 K
Using 250 grains from -204.18 to 183.97 kJ/mol in steps of 1.56 kJ/mol to compute the k(T,P) values at 362.56 K
Using 250 grains from -204.18 to 209.67 kJ/mol in steps of 1.66 kJ/mol to compute the k(T,P) values at 439.853 K
Using 250 grains from -204.18 to 253.55 kJ/mol in steps of 1.84 kJ/mol to compute the k(T,P) values at 571.794 K
Using 255 grains from -204.18 to 327.19 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 791.007 K
Using 307 grains from -204.18 to 435.97 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 1119.5 K
Using 359 grains from -204.18 to 544.76 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 1444.04 K
Saving pressure dependence results for network phenyl_ethene...

Arkane execution terminated at Mon Oct  7 14:28:48 2019


In [12]:
# Retrieve pdep job
pdep_jobs = [job for job in arkane.job_list if isinstance(job, PressureDependenceJob)]
assert len(pdep_jobs) == 1
pdep_job = pdep_jobs[0]

In [13]:
pdep_job.sensitivity_conditions = [[Quantity(condition[0]), Quantity(condition[1])]
                                   for condition in sensitivity_conditions]

In [14]:
# Run pdep sensitivity
perturbation = 0.1  # kcal/mol
for i in range(3):
    try:
        pdep_sens_obj = PDepSensitivity(pdep_job, base_path, perturbation=perturbation, save=True, plot=False)
    except (InvalidMicrocanonicalRateError, ModifiedStrongCollisionError) as e:
        perturbation /= 2.0
    else:
        break
else:
    raise e
sens_coefficients = pdep_sens_obj.sa_coefficients



Perturbing well 'C2H4 + C6H5' by 0.1 kcal/mol:
Calculating densities of states for phenyl_ethene network...
Using 523 grains from -204.18 to 564.51 kJ/mol in steps of 1.47 kJ/mol to compute densities of states
Calculating phenomenological rate coefficients for C2H3 + C6H6 <=> i6...
Using 250 grains from -204.18 to 163.27 kJ/mol in steps of 1.48 kJ/mol to compute the k(T,P) values at 300.312 K
Using 250 grains from -204.18 to 169.67 kJ/mol in steps of 1.50 kJ/mol to compute the k(T,P) values at 319.579 K
Using 250 grains from -204.18 to 183.97 kJ/mol in steps of 1.56 kJ/mol to compute the k(T,P) values at 362.56 K
Using 250 grains from -204.18 to 209.67 kJ/mol in steps of 1.66 kJ/mol to compute the k(T,P) values at 439.853 K
Using 250 grains from -204.18 to 253.55 kJ/mol in steps of 1.84 kJ/mol to compute the k(T,P) values at 571.794 K
Using 255 grains from -204.18 to 327.19 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 791.007 K
Using 307 grains from -204.18 to 435.9



Perturbing well 'i7' by 0.1 kcal/mol:
Calculating densities of states for phenyl_ethene network...
Using 523 grains from -204.18 to 564.51 kJ/mol in steps of 1.47 kJ/mol to compute densities of states
Calculating phenomenological rate coefficients for C2H3 + C6H6 <=> i6...
Using 250 grains from -204.18 to 163.27 kJ/mol in steps of 1.48 kJ/mol to compute the k(T,P) values at 300.312 K
Using 250 grains from -204.18 to 169.67 kJ/mol in steps of 1.50 kJ/mol to compute the k(T,P) values at 319.579 K
Using 250 grains from -204.18 to 183.97 kJ/mol in steps of 1.56 kJ/mol to compute the k(T,P) values at 362.56 K
Using 250 grains from -204.18 to 209.67 kJ/mol in steps of 1.66 kJ/mol to compute the k(T,P) values at 439.853 K
Using 250 grains from -204.18 to 253.55 kJ/mol in steps of 1.84 kJ/mol to compute the k(T,P) values at 571.794 K
Using 255 grains from -204.18 to 327.19 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 791.007 K
Using 307 grains from -204.18 to 435.97 kJ/mol 



Perturbing TS 'TS3' by 0.1 kcal/mol:
Calculating densities of states for phenyl_ethene network...
Using 523 grains from -204.18 to 564.51 kJ/mol in steps of 1.47 kJ/mol to compute densities of states
Calculating phenomenological rate coefficients for C2H3 + C6H6 <=> i6...
Using 250 grains from -204.18 to 163.27 kJ/mol in steps of 1.48 kJ/mol to compute the k(T,P) values at 300.312 K
Using 250 grains from -204.18 to 169.67 kJ/mol in steps of 1.50 kJ/mol to compute the k(T,P) values at 319.579 K
Using 250 grains from -204.18 to 183.97 kJ/mol in steps of 1.56 kJ/mol to compute the k(T,P) values at 362.56 K
Using 250 grains from -204.18 to 209.67 kJ/mol in steps of 1.66 kJ/mol to compute the k(T,P) values at 439.853 K
Using 250 grains from -204.18 to 253.55 kJ/mol in steps of 1.84 kJ/mol to compute the k(T,P) values at 571.794 K
Using 255 grains from -204.18 to 327.19 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 791.007 K
Using 307 grains from -204.18 to 435.97 kJ/mol i



Perturbing TS 'TS6(m)' by 0.1 kcal/mol:
Calculating densities of states for phenyl_ethene network...
Using 523 grains from -204.18 to 564.51 kJ/mol in steps of 1.47 kJ/mol to compute densities of states
Calculating phenomenological rate coefficients for C2H3 + C6H6 <=> i6...
Using 250 grains from -204.18 to 163.27 kJ/mol in steps of 1.48 kJ/mol to compute the k(T,P) values at 300.312 K
Using 250 grains from -204.18 to 169.67 kJ/mol in steps of 1.50 kJ/mol to compute the k(T,P) values at 319.579 K
Using 250 grains from -204.18 to 183.97 kJ/mol in steps of 1.56 kJ/mol to compute the k(T,P) values at 362.56 K
Using 250 grains from -204.18 to 209.67 kJ/mol in steps of 1.66 kJ/mol to compute the k(T,P) values at 439.853 K
Using 250 grains from -204.18 to 253.55 kJ/mol in steps of 1.84 kJ/mol to compute the k(T,P) values at 571.794 K
Using 255 grains from -204.18 to 327.19 kJ/mol in steps of 2.09 kJ/mol to compute the k(T,P) values at 791.007 K
Using 307 grains from -204.18 to 435.97 kJ/mo

In [13]:
# Parse arkane sensitivity output
sens_output = os.path.join(base_path, 'phenyl_ethene.txt')

pdep_sens = {}
with open(sens_output, 'r') as f:
    start = False
    for line in f:
        line = line.strip()
        if 'Sensitivity of' in line:
            if not start:
                start = True
                rxn_sens = {}
            else:
                # Finish previous record
                pdep_sens[rxn] = rxn_sens
                # Initialize new record
                rxn_sens = {}
            rxn = line.split('reaction')[1][:-1].strip()
        elif start and '=====' in line or '-----' in line or 'Well or TS' in line:
            continue
        elif start and line:
            well, temp, pressure, sens = line.strip('|').split('|')
            rxn_sens.setdefault(well.strip(), {})[(float(temp.strip()), float(pressure.strip()))] = float(sens.strip())
    # Finish last record
    pdep_sens[rxn] = rxn_sens

# Calculating overall sensitivities

In [19]:
# Get list of wells and transition states
wells = []
wells.extend(pdep_job.network.reactants)
wells.extend(pdep_job.network.isomers)
wells.extend(pdep_job.network.products)

transition_states = []
for rxn in pdep_job.network.path_reactions:
    transition_states.append(rxn.transition_state)

In [20]:
def get_column(data, label):
    for c in data.columns:
        if label in c:
            return data[c]
    return None

## Calculate sensitivities to species

In [21]:
def calculate_wells(sens_species):
    for well in wells:

        for target_spc in well.species:
            print(f'Calculating for species {target_spc.label}')

            rxn_effect = 0
            for rxn in pdep_job.network.net_reactions:
                rxn_id = rxn.to_chemkin(kinetics=False)
                rxn_str = str(rxn)

                # Retrieve dlnC_n/dk_i
                if get_column(sens_data[sens_species], rxn_id) is not None:
                    dcdk = get_column(sens_data[sens_species], rxn_id).values[-1]
                else:
                    continue

                # Retrieve dk_i/dE_0,m
                dkde = sens_coefficients[rxn_str][well][0]
                if np.isnan(dkde):
                    continue

#                 print(rxn_id)
#                 print(f'dC/dk -> {dcdk!s}')
#                 print(f'dk/dE -> {dkde!s}')

                rxn_effect += dcdk * dkde

            # Retrieve dlnC/dG
            spc_label = f'dG[{target_spc.label}]'
            if get_column(sens_data[sens_species], spc_label) is not None:
                dcdg = get_column(sens_data[sens_species], spc_label).values[-1]
#                 print(f'dC/dG -> {dcdg!s}')
            else:
                dcdg = 0

            total_effect = rxn_effect + dcdg

            print(f'Total Effect = {total_effect!s}')
            print('')

            results[target_spc.label] = total_effect

## Calculate sensitivities to transition states

In [22]:
def calculate_ts(sens_species):
    for ts in transition_states:

        print(f'Calculating for transition state {ts.label}')

        rxn_effect = 0
        for rxn in pdep_job.network.net_reactions:
            rxn_id = rxn.to_chemkin(kinetics=False)
            rxn_str = str(rxn)

            # Retrieve dlnC_n/dk_i
            if get_column(sens_data[sens_species], rxn_id) is not None:
                dcdk = get_column(sens_data[sens_species], rxn_id).values[-1]
            else:
                continue

            # Retrieve dk_i/dE_0,m
            dkde = sens_coefficients[rxn_str][ts][0]
            if np.isnan(dkde):
                continue

#             print(rxn_id)
#             print(f'dC/dk -> {dcdk!s}')
#             print(f'dk/dE -> {dkde!s}')

            rxn_effect += dcdk * dkde

        print(f'Total Effect = {rxn_effect!s}')
        print('')

        results[ts.label] = rxn_effect

In [29]:
all_results = {}
for species in sensitive_species:
    results = {}
    calculate_wells(species)
    calculate_ts(species)
    all_results[species] = results

Calculating for species C2H4
Total Effect = -0.0025334077851870643

Calculating for species C6H5
Total Effect = -0.0025334077851870643

Calculating for species i1
Total Effect = 0.01011482512391792

Calculating for species i2
Total Effect = 0.003814890972089368

Calculating for species i3
Total Effect = -0.011210831154403815

Calculating for species i4
Total Effect = -5.436525171725263e-06

Calculating for species i5
Total Effect = 2.1140120473703446e-06

Calculating for species i6
Total Effect = 1.3516388988459454e-07

Calculating for species i7
Total Effect = 0.00536876314044159

Calculating for species i8
Total Effect = 3.315579671298857e-06

Calculating for species i9
Total Effect = 5.179317178645207e-06

Calculating for species H
Total Effect = -0.005420145081894471

Calculating for species C6H5C2H3
Total Effect = -0.005420145081894471

Calculating for species C2H3
Total Effect = 1.9319389579247547e-07

Calculating for species C6H6
Total Effect = 1.9319389579247547e-07

Calculatin

In [30]:
all_results

{Species(label="C6H5C2H3", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[1.41597,0.0424125,2.36912e-05,-5.44223e-08,2.14136e-11,15248.9,20.2459], Tmin=(100,'K'), Tmax=(1047.78,'K')), NASAPolynomial(coeffs=[14.0832,0.0309758,-1.27931e-05,2.42246e-09,-1.73876e-13,10567.7,-51.1291], Tmin=(1047.78,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Narayanaswamy"""), molecule=[Molecule(smiles="C=Cc1ccccc1"), Molecule(smiles="C=CC1=CC=CC=C1")], molecular_weight=(104.149,'amu')): {'C2H4': -0.0025334077851870643,
  'C6H5': -0.0025334077851870643,
  'i1': 0.01011482512391792,
  'i2': 0.003814890972089368,
  'i3': -0.011210831154403815,
  'i4': -5.436525171725263e-06,
  'i5': 2.1140120473703446e-06,
  'i6': 1.3516388988459454e-07,
  'i7': 0.00536876314044159,
  'i8': 3.315579671298857e-06,
  'i9': 5.179317178645207e-06,
  'H': -0.005420145081894471,
  'C6H5C2H3': -0.005420145081894471,
  'C2H3': 1.9319389579247547e-07,
  'C6H6': 1.9319389579247547e-07,
  'TS1': -6.15173938425

# Run MBSampledReactor simulation

In [31]:
from rmgpy.tools.simulate import run_simulation

In [ ]:
# User specification
mbsamplingRate = 3500
constantSpecies = ['C2H4', 'He']

In [46]:
simulator_input = ''

for spc, molfrac in initial_mole_fractions.items():
    simulator_input += f'''
species(
    label={spc.label!r},
    structure=adjacencyList(
"""
{spc.molecule[0].to_adjacency_list()!s}
"""),
)
'''

mole_fraction_str = {spc.label: molfrac for spc, molfrac in initial_mole_fractions.items()}

simulator_input += f'''
mbsampledReactor(
    temperature={T!s},
    pressure=(10, 'torr'),
    initialMoleFractions={mole_fraction_str!r},
    mbsamplingRate={mbsamplingRate},
    terminationTime={termination_time!s},
    constantSpecies={constantSpecies}
)

simulator(
    atol=1e-10,
    rtol=1e-6,
    sens_atol=1e-6,
    sens_rtol=1e-4,
)
'''

In [48]:
with open(os.path.join(base_path, 'sim_input.py'), 'w') as f:
    f.write(simulator_input)

In [49]:
input_file = os.path.join(base_path, 'sim_input.py')
run_simulation(input_file, chem_path, dict_path)

Removing old /home/mjliu/Documents/new_sensitivity/sensitivity_related/simulate_backup.log
Moving /home/mjliu/Documents/new_sensitivity/sensitivity_related/simulate.log to /home/mjliu/Documents/new_sensitivity/sensitivity_related/simulate_backup.log

Reading input file "/home/mjliu/Documents/new_sensitivity/sensitivity_related/sim_input.py"...

species(
    label='C6H5',
    structure=adjacencyList(
"""
multiplicity 2
1  C u0 p0 c0 {2,B} {3,B} {8,S}
2  C u0 p0 c0 {1,B} {4,B} {7,S}
3  C u0 p0 c0 {1,B} {5,B} {9,S}
4  C u0 p0 c0 {2,B} {6,B} {10,S}
5  C u0 p0 c0 {3,B} {6,B} {11,S}
6  C u1 p0 c0 {4,B} {5,B}
7  H u0 p0 c0 {2,S}
8  H u0 p0 c0 {1,S}
9  H u0 p0 c0 {3,S}
10 H u0 p0 c0 {4,S}
11 H u0 p0 c0 {5,S}

"""),
)
species(
    label='C2H4',
    structure=adjacencyList(
"""
1 C u0 p0 c0 {2,D} {3,S} {4,S}
2 C u0 p0 c0 {1,D} {5,S} {6,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {2,S}
6 H u0 p0 c0 {2,S}

"""),
)
species(
    label='He',
    structure=adjacencyList(
"""
1 He u0 p1 c0

"

In [51]:
num_spc = 2 * len(uncertainty.species_list) - len(constantSpecies)
csv_file = os.path.join(base_path, 'solver', 'simulation_1_{0}.csv'.format(num_spc))
simulation_results = pd.read_csv(csv_file)

In [52]:
simulation_results

,Time (s),Volume (m^3),C6H5,C2H4,C6H6,C2H3,C6H5C2H3,H,i1,i2,...,H_obs,i1_obs,i2_obs,i3_obs,i4_obs,i5_obs,i6_obs,i7_obs,i8_obs,i9_obs
0,0.000000e+00,3.785475,1.115921e-05,0.299439,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1.000000e-15,3.785475,1.115921e-05,0.299439,5.713014e-19,5.713014e-19,2.478767e-19,2.478767e-19,2.230618e-17,4.750045e-19,...,8.675686e-31,7.807163e-29,1.662516e-30,1.346341e-30,7.043139e-32,2.070392e-33,-9.578001e-61,-6.048432e-57,-2.973498e-59,-4.347858e-60
2,3.000000e-15,3.785475,1.115921e-05,0.299439,1.877133e-18,1.877133e-18,8.144521e-19,8.144521e-19,7.329173e-17,1.560729e-18,...,5.701165e-30,5.130421e-28,1.092510e-29,8.847385e-30,4.628348e-31,1.360543e-32,2.855156e-59,1.803009e-55,8.863855e-58,1.296076e-58
3,7.000000e-15,3.785475,1.115921e-05,0.299439,4.162339e-18,4.162339e-18,1.805959e-18,1.805959e-18,1.625165e-16,3.460747e-18,...,3.098459e-29,2.788272e-27,5.937556e-29,4.808361e-29,2.515405e-30,7.394254e-32,4.231913e-58,2.672420e-54,1.313801e-56,1.921044e-57
4,1.500000e-14,3.785475,1.115921e-05,0.299439,8.732750e-18,8.732750e-18,3.788973e-18,3.788973e-18,3.409659e-16,7.260783e-18,...,1.370758e-28,1.233532e-26,2.626775e-28,2.127219e-28,1.112814e-29,3.271217e-31,5.986720e-57,3.780567e-53,1.858582e-55,2.717624e-56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,7.600483e-03,3.785478,3.712802e-09,0.299428,2.668554e-07,2.668554e-07,1.992399e-07,1.992399e-07,1.031720e-05,3.686803e-07,...,1.959187e-07,1.032647e-05,3.628232e-07,2.498375e-09,2.560641e-13,6.605092e-18,6.100493e-15,9.684012e-10,6.719401e-13,8.834554e-13
136,8.528328e-03,3.785478,3.699874e-09,0.299428,2.670312e-07,2.670312e-07,2.099737e-07,2.099737e-07,1.028720e-05,3.876283e-07,...,2.066646e-07,1.029644e-05,3.817883e-07,2.491110e-09,2.553194e-13,6.582218e-18,6.535544e-15,1.133169e-09,7.497273e-13,1.057095e-12
137,9.456173e-03,3.785478,3.688403e-09,0.299428,2.672064e-07,2.672064e-07,2.206675e-07,2.206675e-07,1.025728e-05,4.065213e-07,...,2.173707e-07,1.026650e-05,4.006983e-07,2.483865e-09,2.545768e-13,6.561851e-18,6.991596e-15,1.305866e-09,8.312798e-13,1.242677e-12
138,1.000000e-02,3.785479,3.681833e-09,0.299428,2.673089e-07,2.673089e-07,2.269205e-07,2.269205e-07,1.023977e-05,4.175749e-07,...,2.236310e-07,1.024898e-05,4.117618e-07,2.479626e-09,2.541424e-13,6.550179e-18,7.268149e-15,1.410277e-09,8.807181e-13,1.356029e-12


In [64]:
cn_label = sensitive_species[0].label
cn_obs_label = cn_label + '_obs'

cn = simulation_results[cn_label].values[-1]
cn_obs = simulation_results[cn_obs_label].values[-1]

In [73]:
sampling_factor = (cn / cn_obs) * (cn_obs - simulation_results[cn_obs_label].values[-2]) / (cn - simulation_results[cn_label].values[-2])
print(sampling_factor)

1.0149003474110512


In [71]:
signal_results = {}

for species in sensitive_species:
    new_results = {}
    for item, value in all_results[species].items():
        new_results[item] = value * sampling_factor
    signal_results[species] = new_results

In [72]:
signal_results

{Species(label="C6H5C2H3", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[1.41597,0.0424125,2.36912e-05,-5.44223e-08,2.14136e-11,15248.9,20.2459], Tmin=(100,'K'), Tmax=(1047.78,'K')), NASAPolynomial(coeffs=[14.0832,0.0309758,-1.27931e-05,2.42246e-09,-1.73876e-13,10567.7,-51.1291], Tmin=(1047.78,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Narayanaswamy"""), molecule=[Molecule(smiles="C=Cc1ccccc1"), Molecule(smiles="C=CC1=CC=CC=C1")], molecular_weight=(104.149,'amu')): {'C2H4': -0.0025711564413202132,
  'C6H5': -0.0025711564413202132,
  'i1': 0.010265539532266325,
  'i2': 0.0038717341729087823,
  'i3': -0.011377876433371069,
  'i4': -5.517531285492894e-06,
  'i5': 2.1455115613073103e-06,
  'i6': 1.3717787880130407e-07,
  'i7': 0.005448759576401816,
  'i8': 3.364982960270229e-06,
  'i9': 5.256490803959046e-06,
  'H': -0.0055009071266329986,
  'C6H5C2H3': -0.0055009071266329986,
  'C2H3': 1.9607255195747778e-07,
  'C6H6': 1.9607255195747778e-07,
  'TS1': -6.243402